In [14]:
%pip install -r requirements.txt

  Using cached catboost-1.2-cp310-cp310-win_amd64.whl (101.0 MB)
  Using cached scikit_learn-1.2.2-cp310-cp310-win_amd64.whl (8.3 MB)
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0
Note: you may need to restart the kernel to use updated packages.


In [26]:
import numpy as np

import pandas as pd

from catboost import CatBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df = pd.read_csv('df_3.csv', index_col=0)

In [7]:
df = df.drop(['date', 'day', 'month', 'year', 'dayofweek', 'street', 'lat_long', 'price_sq_meter'], axis=1)

In [17]:
df

,rooms,district,floor,total_floors,total_area,living_area,kitchen_area,price,park_eko,attraction,south,eternal_fire,veter,city_center,floor_type
0,2,Орджоникидзевский,8.0,9.0,50.0,30.0,9.0,3600.0,4.66,2.70,1.49,4.65,7.30,4.76,middle
1,2,Правобережный,5.0,5.0,44.0,27.0,6.0,2900.0,2.67,1.19,3.49,3.06,5.68,2.96,last
2,1,NaN,1.0,1.0,32.0,17.0,7.0,1600.0,NaN,NaN,NaN,NaN,NaN,NaN,first
3,5,Ленинский,3.0,3.0,149.0,100.0,12.0,12750.0,3.32,5.21,7.96,1.95,1.04,1.87,last
4,2,NaN,5.0,5.0,44.0,29.0,6.0,2930.0,NaN,NaN,NaN,NaN,NaN,NaN,last
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,3,Ленинский,5.0,5.0,74.0,56.0,9.0,5500.0,4.32,6.22,8.82,2.81,0.34,2.86,last
464,2,Ленинский,1.0,5.0,41.0,22.2,7.2,930.0,12.63,14.30,15.76,10.60,8.28,10.96,first
465,2,Правобережный,5.0,5.0,42.5,29.0,6.0,2650.0,1.74,3.41,6.40,1.03,2.77,0.31,last
466,2,Орджоникидзевский,3.0,9.0,50.0,30.0,9.0,3350.0,5.87,4.29,1.65,5.06,7.53,5.41,middle


In [41]:
df.isin([None]).sum()

rooms           0
district        0
floor           0
total_floors    0
total_area      0
living_area     0
kitchen_area    0
price           0
park_eko        0
attraction      0
south           0
eternal_fire    0
veter           0
city_center     0
floor_type      0
dtype: int64

In [50]:
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=47,
                                                    shuffle=True,
                                                    #stratify=df['price']
                                                    )

In [23]:
categorical = ['district', 'floor', 'total_floors', 'floor_type']
print(f"Категориальные признаки: {categorical}")

numerical = list(set(X_train.columns) - set(categorical))
print(f"Числовые признаки: {numerical}")

Категориальные признаки: ['district', 'floor', 'total_floors', 'floor_type']
Числовые признаки: ['park_eko', 'veter', 'total_area', 'rooms', 'eternal_fire', 'living_area', 'south', 'attraction', 'city_center', 'kitchen_area']


In [45]:
df[categorical] = df[categorical].astype(str)

In [49]:
df[categorical].isna()

,district,floor,total_floors,floor_type
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
463,False,False,False,False
464,False,False,False,False
465,False,False,False,False
466,False,False,False,False


In [55]:
pipe = Pipeline(
    [
        ("regressor", CatBoostRegressor())
    ]
)

param_grid = [
    {
        'regressor': [CatBoostRegressor(random_state=47, 
                                        eval_metric='RMSE',
                                        verbose=100,
                                        #silent=True,
                                        cat_features=categorical,
                                        early_stopping_rounds=20)],
        # 'regressor__depth': range(2, 12, 2),
        # 'regressor__learning_rate': np.linspace(0.3, 0.9, 7),
        # 'regressor__iterations': range(50,1000,50)
    }
]

grid = RandomizedSearchCV(pipe, param_grid, 
                          n_iter=50 ,
                          cv=5, 
                          scoring = 'neg_mean_squared_error', 
                          verbose=1)

In [56]:
%%time
grid.fit(X_train, y_train)

e:\GITlocal\Magnitogorsk_real_estate\.venv\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=50. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Learning rate set to 0.033407
0:	learn: 1525.9931495	total: 27.1ms	remaining: 27s
100:	learn: 783.2141695	total: 2.63s	remaining: 23.4s
200:	learn: 607.4599061	total: 5.26s	remaining: 20.9s
300:	learn: 494.5508042	total: 8.18s	remaining: 19s


KeyboardInterrupt: 